# BB84  protocol

BB84 is a simple qunatum cryptography protocol meeant to take advantage of two maximally incompatible bases to safely distribute keys. The basic procedure is as follows

* Alice prepares a set of $N$ qubits by measuring their state in one of two maximally incompatible bases chosen at random. Typically, the chosen bases are the Hadamard and computational bases. Through this process, she generates a pseudo-key.
* Alice sends her qubits to Bob through a quantum communication channel.
* Once Bob receives the qubits, he measures them. However, because he does not know which of the two bases Alice used for each of the qubits, he measures each qubit by randomly selecting between the two incompatible bases. After this, he has his own pseudo-key.
* Alice and Bob exchange the bases they used for each qubit through a classical channel and throw away all the bits in their respective pseudo-keys for which they did not match.
* To check if there was any tampering of the qubits when they were sent, Alice and Bob choose a mutually agreed subset of their keys' bits through the classical channel. If any qubit does not match, they will have uncovered the intruder.

If Eve had intecepted the qubits as Alice was sending them to Bob, she would be in the same position as Bob: because she does not know the basis Alice used for each of the qubits, she would have to random select one of the two. This means that, as Alice and Bob are comparing their check bits, there is a chance that they will find a discrepancy, thus revealing Eve's presence. The probability of this happening is given by the following considerations:
* There is a $50\%$ chance that Eve chose the wrong bases for a particular qubit.
* There is a $50\%$ chance that Bob's measurement does not match Alice's for a particular qubit if the qubit was tampered with by Eve.
* Therefore, there is a $25\%$ chance that Eve will be discovered if Alice and Bob compare that particular qubit.

Thus, the probability of Eve being discovered if Alice and Bob compare $n$ bits is

$$
    P(n) = 1 - \left(\frac34\right)^n
$$

The following function implements this protocol using `qcrypto`.

In [ ]:
from qcrypto.simbasics import QstateUnEnt, QstateEnt, Agent
from qcrypto.gates import H_gate
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

In [ ]:
def BB84(numqubits: int, numcheckbits: int, eve: bool = False):
    """
    Implements the BB84 protocol using qcrypto.

    Args:
        - numqubits (int): Number of qubits Alice will send to Bob
        - numcheckbits (int): Number of bits Alice and Bob will compare through the classical channel
        - eve (bool): Whether or not Eve will intrude in Alice and Bob's key exchange

    Returns:
        Whether or not Eve was discovered, as well as Alice's and Bob's keys at the end of the protocol
    """

    # Initializations

    public_qstate = QstateUnEnt(init_method="random", num_qubits=numqubits)

    Alice = Agent()
    Alice.set_qstate(qstate=public_qstate, qstate_type="public")
    Bob = Agent()
    Bob.set_qstate(qstate=public_qstate, qstate_type="public")
    if eve:
        Eve = Agent()
        Eve.set_qstate(qstate=public_qstate, qstate_type="public")

    # Protocol
    
    Alice_base_choice = np.random.choice([True, False], size=(numqubits))
    Alice_to_gate = np.nonzero(Alice_base_choice)[0]
    Alice.apply_gate(qstate_type="public", gate=H_gate, qubit_idx=Alice_to_gate)
    Alice.get_key(qstate_type="public")
    Alice.apply_gate(qstate_type="public", gate=H_gate, qubit_idx=Alice_to_gate)

    if eve:
        Eve_base_choice = np.random.choice([True, False], size=(numqubits))
        Eve_to_gate = np.nonzero(Eve_base_choice)[0]
        Eve.apply_gate(qstate_type="public", gate=H_gate, qubit_idx=Eve_to_gate)
        Eve.get_key(qstate_type="public")
        Eve.apply_gate(qstate_type="public", gate=H_gate, qubit_idx=Eve_to_gate)

    Bob_base_choice = np.random.choice([True, False], size=(numqubits))
    Bob_to_gate = np.nonzero(Bob_base_choice)[0]
    Bob.apply_gate(qstate_type="public", gate=H_gate, qubit_idx=Bob_to_gate)
    Bob.get_key(qstate_type="public")
    Bob.apply_gate(qstate_type="public", gate=H_gate, qubit_idx=Bob_to_gate)

    # Throwing away bits with diff bases
    diff_base_mask = Alice_base_choice == Bob_base_choice

    Alice.keys["public"] = Alice.keys["public"][diff_base_mask]
    Bob.keys["public"] = Bob.keys["public"][diff_base_mask]

    # Alice and Bob communicate check bits 
    Alice_checkbits = Alice.keys["public"][:numcheckbits]
    Bob_checkbits = Bob.keys["public"][:numcheckbits]
    
    Alice.keys["public"] = Alice.keys["public"][numcheckbits:]
    Bob.keys["public"] = Bob.keys["public"][numcheckbits:]

    discovered = ~np.all(Alice_checkbits == Bob_checkbits)

    return discovered, (Alice.keys["public"], Bob.keys["public"])

In [ ]:
def run_trial(num_checks, num_qubits, eve_present=True):
    detected, _ = BB84(num_qubits, num_checks, eve=eve_present)
    return detected

In [ ]:
numqubits = 25
nchecks_lst = np.arange(numqubits//2)
probs = [] 
numtrials = 1000

for nchecks in nchecks_lst:
    detections = 0
    for _ in range(numtrials):
        detected = run_trial(nchecks, numqubits, eve_present=True)
        detections += detected
    prob = detections / numtrials
    probs.append(prob)   

In [ ]:
def prob_func(x, p):
    return 1 - (p)**x

param, param_cov = curve_fit(prob_func, nchecks_lst, probs)

plt.errorbar(y=probs, x=nchecks_lst, yerr=[1/np.sqrt(numtrials)]*len(probs), fmt=".", capsize=3, label="Computed Probability")
plt.plot(nchecks_lst, prob_func(nchecks_lst, param), label=r"P = 1 - ({})$^n$".format(round(param[0], 3)))
plt.plot(nchecks_lst, prob_func(nchecks_lst, 3/4), label=r"P = 1 - $(3/4)^n$".format(round(param[0], 3)))

plt.title(r"Probability of Eve being detected (Eve)")
plt.xlabel(r"Number of checkbits ($n_{\mathrm{check}}$)")
plt.ylabel(r"$P_{\mathrm{detection}}(n_{\mathrm{check}}\ |\ \mathrm{Eve})$")
plt.ylim(0, 1)
plt.legend()
plt.grid()
plt.savefig("../plots_BB84/Eve.png")
plt.show()

In [ ]:
numqubits = 25
nchecks_lst = np.arange(numqubits//3)
probs = [] 
numtrials = 1000

def run_trial(num_checks, num_qubits, eve_present=True):
    detected, _ = BB84(num_qubits, num_checks, eve=eve_present)
    return detected

for nchecks in nchecks_lst:
    detections = 0
    for _ in range(numtrials):
        detected = run_trial(nchecks, numqubits, eve_present=False)
        detections += detected
    prob = detections / numtrials
    probs.append(prob)   

In [ ]:
plt.errorbar(y=probs, x=nchecks_lst, yerr=[1/np.sqrt(numtrials)]*len(probs), fmt=".", capsize=3, label="Computed Probability")

plt.title(r"Probability of Eve being detected (No Eve)")
plt.xlabel(r"Number of checkbits ($n_{\mathrm{check}}$)")
plt.ylabel(r"$P_{\mathrm{detection}}(n_{\mathrm{check}}\ |\ \mathrm{\text{No Eve}})$")
plt.ylim(0, 1)
plt.legend()
plt.grid()
plt.savefig("../plots_BB84/NoEve.png")
plt.show()